# Comparison of different reference databases

In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from util import *

# set path to data folder
root = "F:/Studium/Master/semester5/thesis/data/dataset/"

# definition of the true positive species
species = {
    "Pseudomonas aeruginosa",
    "Escherichia coli",
    "Salmonella enterica",
    "Staphylococcus aureus",
    "Limosilactobacillus fermentum",
    "Enterococcus faecalis",
    "Listeria monocytogenes",
    "Bacillus subtilis",
    "Saccharomyces cerevisiae",
    "Cryptococcus neoformans"}

# total read count in zymoMock dataset
total = 1_160_526
# reading the taxonomic tree
tree = Tree(root + "tree_nr.txt")
# generate dictionary with all taxa that are true positives for each taxonomic rank
true_taxons = get_true_taxons_for_all_ranks(tree, species)

In [ ]:
# reading in datasets
data_nr50 = read_per_taxon_assignment(root + "zymo_mock/assignment_nr50_uniform11s_13-mer/per_taxon_assignments.tsv", kmer_threshold=1, ovo_1_threshold=0)
data_nr90 = read_per_taxon_assignment(root + "zymo_mock/assignment_nr90_uniform11s_13-mer/per_taxon_assignments.tsv", kmer_threshold=1, ovo_1_threshold=0)
data_nr = read_per_taxon_assignment(root + "zymo_mock/assignment_nr_uniform11s_13-mer/per_taxon_assignments.tsv", kmer_threshold=1, ovo_1_threshold=0)

# add column with labels for the true positives
data_nr50 = classify_assigned_taxa(data_nr50, true_taxons)
data_nr90 = classify_assigned_taxa(data_nr90, true_taxons)
data_nr = classify_assigned_taxa(data_nr, true_taxons)

# transform algorithm columns into one column with algorithm, parameter and used data
data_nr50 = extract_algorithm_info(data_nr50)
data_nr90 = extract_algorithm_info(data_nr90)
data_nr = extract_algorithm_info(data_nr)

# calculate the precision and recall for each algorithm-parameter-data combination and each rank
data_nr50 = get_precision_recall_reads(data_nr50, total)
data_nr90 = get_precision_recall_reads(data_nr90, total)
data_nr = get_precision_recall_reads(data_nr, total)

data_nr

In [ ]:
data_nr50_filtered = data_nr50[(data_nr50["algorithm"] == "OVA") & (data_nr50["algorithm parameter"] == 0.5) & (data_nr50["algorithm data"] == "kmer count (cumulative)")]
data_nr90_filtered = data_nr90[(data_nr90["algorithm"] == "OVA") & (data_nr90["algorithm parameter"] == 0.5) & (data_nr90["algorithm data"] == "kmer count (cumulative)")]
data_nr_filtered = data_nr[(data_nr["algorithm"] == "OVA") & (data_nr["algorithm parameter"] == 0.5) & (data_nr["algorithm data"] == "kmer count (cumulative)")]
data_nr90_filtered

In [ ]:
colors = ["red", "lightcoral", "gold", "yellow", "royalblue", "cornflowerblue", "lightsteelblue", "lavender", "darkgreen", "seagreen", "mediumseagreen", "darkseagreen", "mintcream"]

plt.figure(figsize=(5, 5), dpi=200)

for i, rank in enumerate(["species", "genus", "family", "order"]):
    data_filtered = data_nr50_filtered[data_nr50_filtered["rank"] == rank]
    plt.scatter(
        data_filtered["recall"],
        data_filtered["precision"],
        label="nr50" + " " + rank,
        color=colors[i],
    )
for i, rank in enumerate(["species", "genus", "family", "order"]):
    data_filtered = data_nr90_filtered[data_nr90_filtered["rank"] == rank]
    plt.scatter(
        data_filtered["recall"],
        data_filtered["precision"],
        label="nr90" + " " + rank,
        color=colors[i+4],
    )
for i, rank in enumerate(["species", "genus", "family", "order"]):
    data_filtered = data_nr_filtered[data_nr_filtered["rank"] == rank]
    plt.scatter(
        data_filtered["recall"],
        data_filtered["precision"],
        label="nr" + " " + rank,
        color=colors[i+8],
    )
plt.xlim(0, 1)
plt.xticks(np.arange(0, 1.01, 0.1))
plt.ylim(0, 1)
plt.yticks(np.arange(0, 1.01, 0.1))
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Different Reference Databases for DIAMER")
plt.legend(loc="lower right")
plt.show()